In [1]:
from typing import List
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import scipy.stats as st
import matplotlib.pyplot as plt

In [17]:
N_DIMS = 2
N_PARTICLES = 10**3
STEPS = 10**3
LOW, HIGH = -4.5, 4.5
MIN_VEL, MAX_VEL = 0, 10

def sphere(vector):
    return np.sum(np.power(vector, 2))
    
def rastrigin(vector):
    A = 10
    return A + np.sum(np.power(vector, 2) - A * np.cos(2*np.math.pi*vector))

def rosenbrock(vector):
    return np.power(np.math.e, -np.sum(np.array([(100*(vector[i+1]-vector[i]**2)**2 + (1-vector[i])**2)/20 for i in range(len(vector) - 1)])))

target_func = lambda vector: rastrigin(vector)

In [18]:
class Particle():
    def __init__(self, calc_objective=True):
        self.velocity=np.random.uniform(low=MIN_VEL, high=MAX_VEL, size=N_DIMS)
        self.position=np.random.uniform(low=LOW, high=HIGH, size=N_DIMS)
        self.local_best_position=self.position
        self.local_best_value=objective_function(self.local_best_position) if calc_objective else 0
        self.position_trace=np.empty(shape=(STEPS, N_DIMS))
        self.velocity_trace=np.empty(shape=(STEPS, N_DIMS))

def initialize():
    particles = [Particle() for _ in np.arange(0, N_PARTICLES)]
    return particles

def valid_pos(position):
    for pos in position:
        if pos < LOW or pos > HIGH:
            return False
    return True

def fix_position(position):
    fixed_pos = []
    for pos in position:
        if pos < LOW:
            fixed_pos.append(LOW)
        elif pos > HIGH:
            fixed_pos.append(HIGH)
        else:
            fixed_pos.append(pos)
    return np.float32(np.array(fixed_pos))

def objective_function(position):
    if valid_pos(position) is False:
        return np.finfo(np.float32).min
    return target_func(vector=position)

def pso(particles: List[Particle]):
    """
    Returns local_bests & global_best
    (particles, local_bests) -> (local_bests, global_best)
    """
    inertia_weight = st.norm(loc=1, scale=.5).rvs(size=1)
    c1, c2 = st.norm(loc=2, scale=.5).rvs(size=2)

    swarm_best_position = particles[0].local_best_position

    for i in np.arange(1, len(particles)):
        if objective_function(swarm_best_position) > objective_function(particles[i].local_best_position):
            swarm_best_position = particles[i].local_best_position
    
    for i in np.arange(STEPS):
        for j in np.arange(N_PARTICLES):
            particles[j].velocity = inertia_weight * particles[j].velocity +\
                c1*np.random.random()*(particles[j].local_best_position - particles[j].position) +\
                    c2*np.random.random()*(swarm_best_position - particles[j].position)

            _position = particles[j].position
            _velocity = particles[j].velocity

            particles[j].position += particles[j].velocity
            particles[j].position = np.float32(fix_position(particles[j].position))

            candidate = objective_function(particles[j].position)

            if candidate > objective_function(particles[j].local_best_position):
                particles[j].local_best_position = particles[j].position
            
            if objective_function(particles[j].local_best_position) > objective_function(swarm_best_position):
                swarm_best_position = particles[j].local_best_position

            particles[j].position_trace[i] = particles[j].position
            particles[j].velocity_trace[i] = particles[j].velocity

    return swarm_best_position, particles

In [19]:
particles = initialize()
swarm_best_position, particles = pso(particles=particles)
print(f"Swarm Best: -> {swarm_best_position}")
best_locals = np.array([particle.local_best_position for particle in particles])
best_unique_locals = np.unique(best_locals, axis=0)
print("====================")
print(f"Best Unique Locals: \n {best_unique_locals}")
# for particle in particles:
#     print(f"\t -> {particle.local_best_position}")
pass

Swarm Best: -> [4.5 4.5]
Best Unique Locals: 
 [[4.5 4.5]]
